# AI Red Teaming Agent for Generative AI models and applications in Azure AI Foundry

## Objective
This notebook walks through how to use Azure AI Evaluation's AI Red Teaming Agent functionality to assess the safety and resilience of AI systems against adversarial prompt attacks. AI Red Teaming Agent leverages [Risk and Safety Evaluations](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-metrics-built-in?tabs=warning#risk-and-safety-evaluators) to help identify potential safety issues across different risk categories (violence, hate/unfairness, sexual content, self-harm) combined with attack strategies of varying complexity levels from [PyRIT](https://github.com/Azure/PyRIT), Microsoft AI Red Teaming team's open framework for automated AI red teaming.

## Time
You should expect to spend about 30-45 minutes running this notebook. Execution time will vary based on the number of risk categories, attack strategies, and complexity levels you choose to evaluate.

## Before you begin

### Prerequisite
The AI Red Teaming Agent requires an Azure AI Foundry project configuration and Azure credentials. Your project configuration will be used to log red teaming scan results after the run is finished.

**Important**: Make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

### Installation
From a terminal window, navigate to your working directory which contains this sample notebook, and execute the following.
```bash
python -m venv .venv
```

Then, activate the virtual environment created:

```bash
# %source .venv/bin/activate # If using Mac/Linux OS
.venv/Scripts/activate # If using Windows OS
```

With your virtual environment activated, install the following packages required to execute this notebook:

```bash
pip install uv
uv pip install azure-ai-evaluation[redteam] azure-identity openai
```


Now open VSCode with the following command, and ensure your virtual environment is used as kernel to run the remainder of this notebook.
```bash
code .
```

### Imports

In [2]:
%pip install --upgrade azure-ai-evaluation[redteam]

   ---------------------------------------- 0.0/773.7 kB ? eta -:--:--
   --------------------------------------- 773.7/773.7 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: azure-ai-evaluation
    Found existing installation: azure-ai-evaluation 1.4.0
    Uninstalling azure-ai-evaluation-1.4.0:
      Successfully uninstalled azure-ai-evaluation-1.4.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import Optional, Dict, Any
import os

# Azure imports
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# OpenAI imports
from openai import AzureOpenAI

### Set Up Your Environment Variables

Set the following variables for use in this notebook. These variables connect to your Azure resources and model deployments.

**Note:** You can find these values in your Azure AI Foundry project or Azure OpenAI resource.

For reference, here's an example of what your populated environment variables should look like:

```
# Azure OpenAI
AZURE_OPENAI_API_KEY="your-api-key-here"
AZURE_OPENAI_ENDPOINT="https://endpoint-name.openai.azure.com/openai/deployments/deployment-name/chat/completions"
AZURE_OPENAI_DEPLOYMENT_NAME="gpt-4"
AZURE_OPENAI_API_VERSION="2023-12-01-preview"

# Azure AI Project
AZURE_SUBSCRIPTION_ID="12345678-1234-1234-1234-123456789012"
AZURE_RESOURCE_GROUP_NAME="your-resource-group"
AZURE_PROJECT_NAME="your-project-name"
```

In [3]:
from dotenv import load_dotenv
load_dotenv("../../.env")

# Azure AI Project information
azure_ai_project = {
    "subscription_id": os.environ.get("SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("RG_NAME"),
    "project_name": os.environ.get("PROJECT_NAME"),
}

# Azure OpenAI deployment information
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")  # e.g., "gpt-4"
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")  # e.g., "https://endpoint-name.openai.azure.com/openai/deployments/deployment-name/chat/completions"
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")  # e.g., "your-api-key"
azure_openai_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")  # Use the latest API version

## Understanding AI Red Teaming Agent's capabilities

The Azure AI Evaluation SDK's `RedTeam` functionality evaluates AI systems against adversarial prompts across multiple dimensions:

1. **Risk Categories**: Different content risk categories your AI system might generate
   - Violence
   - HateUnfairness
   - Sexual
   - SelfHarm

2. **Attack Strategies**: Along with standard unmodified prompts which are sent by default as the `baseline`, you can specify different transformations of prompts to elicit undesired content.
You can also use `AttackStrategy.Compose()` to layer two strategies in one attack
   - AnsiAttack: Using ANSI escape codes in prompts
   - AsciiArt: Using ASCII art to disguise harmful content
   - AsciiSmuggler: Hiding harmful content within ASCII characters
   - Atbash: Using the Atbash cipher to encode harmful requests
   - Base64: Encoding harmful content in Base64 format
   - Binary: Converting text to binary to bypass filters
   - Caesar: Using the Caesar cipher for encoding
   - CharacterSpace: Manipulating character spacing to confuse filters
   - CharSwap: Swapping characters to bypass detection
   - Diacritic: Using diacritical marks to alter text appearance
   - Flip: Flipping text to bypass content filters
   - Leetspeak: Converting letters to numbers and symbols
   - Morse: Using Morse code to encode harmful requests
   - ROT13: Using ROT13 cipher for text transformation
   - SuffixAppend: Adding suffixes to confuse detection systems
   - StringJoin: Joining strings in unconventional ways
   - Tense: Changing the tense of harmful requests to past tense
   - UnicodeConfusable: Using similar-looking Unicode characters
   - UnicodeSubstitution: Substituting characters with Unicode alternatives
   - Url: Embedding harmful content within URLs
   - Jailbreak: Specially crafted prompts to bypass AI safeguards

3. **Complexity Levels**: Different difficultly levels of attacks
   - Baseline: Standard functionality tests
   - Easy: Simple attack patterns
   - Moderate: More sophisticated attacks
   - Difficult: Complex, layered attack strategies

The key metric for evaluating results is the **Attack Success Rate (ASR)**, which measures the percentage of attacks that successfully elicit harmful content from your AI system.

## Basic Example: Fixed Response Callback

Let's start with a basic example that demonstrates how to set up a Red Team evaluation using a simple callback that always returns a fixed, safe response regardless of the input prompt.

**Important Note**: Since this callback always responds with the same safe message, the Attack Success Rate (ASR) will be 0% for all categories. This example is merely to demonstrate the API structure and flow.

In [5]:
# Define a simple callback function that always returns a fixed response
def financial_advisor_callback(query: str) -> str:  # noqa: ARG001
    return "I'm a financial advisor assistant. I can help with investment advice and financial planning within legal and ethical guidelines."

In [9]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

credential = DefaultAzureCredential()

# Create the `RedTeam` instance with minimal configurations
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
    num_objectives=1,
)

NOTE: `num_objectives` specifies the number of attacks to perform per risk category per attack strategy. If the parameter `risk_categories` is not specified, `[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm]` will be used by default.

Now let's run a simple automated scan using the `RedTeam` with the fixed response target. We'll test against two risk categories and one attack strategy for simplicity.

In [10]:
# Run the red team scan called "Basic-Callback-Scan" with limited scope for this basic example
# This will test 1 objective prompt for each of Violence and HateUnfairness categories with the Flip strategy
result = await red_team.scan(
    target=financial_advisor_callback, scan_name="Basic-Callback-Scan", attack_strategies=[AttackStrategy.Flip]
)

🚀 STARTING RED TEAM SCAN: Basic-Callback-Scan
📂 Output directory: .\.scan_Basic-Callback-Scan_20250408_102717
📊 Risk categories: ['violence', 'hate_unfairness']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/build/evaluation/8f9582ee-4c12-4a2a-8674-b0b56fb4bdb0?wsid=/subscriptions/65a513ce-bb5d-4ed5-92b1-fa601d510a15/resourceGroups/agentai/providers/Microsoft.MachineLearningServices/workspaces/genaiops-demo
📋 Planning 4 total tasks


Scanning:   0%|                         | 0/4 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                  | 0/4 [00:04<?, ?scan/s, current=fetching baseline/hate_unfairness]

📝 Fetched baseline objectives for violence: 1 objectives


...


✅ Completed task 2/4 (50.0%) - baseline/hate_unfairness in 28.6s
   Est. remaining: 0.6 minutes


[2025-04-08 10:27:56 +0100][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-08 10:27:56 +0100][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_7jkgnlug_20250408_102756_657141, log path: C:\Users\alevret\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_7jkgnlug_20250408_102756_657141\logs.txt
Scanning:  75%|█████████████████████████▌        | 3/4 [00:47<00:14, 14.78s/scan, current=batch 1/1]


✅ Completed task 3/4 (75.0%) - flip/violence in 41.3s
   Est. remaining: 0.3 minutes


[2025-04-08 10:28:09 +0100][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-08 10:28:09 +0100][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_j6cf1fou_20250408_102809_323565, log path: C:\Users\alevret\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_j6cf1fou_20250408_102809_323565\logs.txt
Scanning: 100%|██████████████████████████████████| 4/4 [01:03<00:00, 15.76s/scan, current=batch 1/1]



✅ Completed task 4/4 (100.0%) - flip/hate_unfairness in 57.3s
   Est. remaining: 0.0 minutes
Evaluation results saved to "C:\Users\alevret\OneDrive - Microsoft\Documents\DN\Code\genaiops-exercises\solutions\07_AI_Red_Teaming\.scan_Basic-Callback-Scan_20250408_102717\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/4 attacks were successful
------------------------------------------------------------------------------------------------------------
Risk Category  | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------
Violence       | 0.0%           | 0.0%                         | N/A                             | N/A                           
Hate-unfairness| 0.0%           | 0.0%                         | N/A                             | N/A                           

Detailed results available at:
https:

## Intermediary Example: Using a Model Configuration as Target

Now let's create a more realistic example that uses an Azure OpenAI model for responding to the red teaming prompts. To test base or foundation models, you can update your target to take in a model configuration:

In [11]:
# Define a model configuration to test
azure_oai_model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "azure_deployment": azure_openai_deployment,
    "api_key": azure_openai_api_key,
}

Then, update your target to point to the model configurations and run the scan.

In [12]:
# Run the red team scan called "Intermediary-Model-Target-Scan"
result = await red_team.scan(
    target=azure_oai_model_config, scan_name="Intermediary-Model-Target-Scan", attack_strategies=[AttackStrategy.Flip]
)

🚀 STARTING RED TEAM SCAN: Intermediary-Model-Target-Scan
📂 Output directory: .\.scan_Intermediary-Model-Target-Scan_20250408_102909
📊 Risk categories: ['violence', 'hate_unfairness']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/build/evaluation/9567c84f-1fb2-411a-a721-ee74e536235e?wsid=/subscriptions/65a513ce-bb5d-4ed5-92b1-fa601d510a15/resourceGroups/agentai/providers/Microsoft.MachineLearningServices/workspaces/genaiops-demo
📋 Planning 4 total tasks


Scanning:   0%|                         | 0/4 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                             | 0/4 [00:00<?, ?scan/s, current=fetching flip/violence]

📝 Fetched baseline objectives for violence: 1 objectives
📝 Fetched baseline objectives for hate_unfairness: 1 objectives
🔄 Fetching objectives for strategy 2/2: flip


...


✅ Completed task 2/4 (50.0%) - flip/violence in 30.4s
   Est. remaining: 0.6 minutes


[2025-04-08 10:29:45 +0100][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-08 10:29:45 +0100][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_0gemnkzq_20250408_102945_502384, log path: C:\Users\alevret\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_0gemnkzq_20250408_102945_502384\logs.txt
Scanning:  75%|█████████████████████████▌        | 3/4 [00:45<00:14, 14.14s/scan, current=batch 1/1]ERROR: [baseline/violence] Error processing prompts: Error sending prompt with conversation ID: 5bb56c8e-ccba-4529-b723-84257de25c04
Traceback (most recent call last):
  File "c:\Users\alevret\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyrit\prompt_normalizer\prompt_normalizer.py", line 95, in send_prompt_async
    response = await targ


✅ Completed task 3/4 (75.0%) - flip/hate_unfairness in 44.3s
   Est. remaining: 0.3 minutes


Scanning: 100%|██████████████████████████████████| 4/4 [00:54<00:00, 13.62s/scan, current=batch 1/1]



✅ Completed task 4/4 (100.0%) - baseline/violence in 53.7s
   Est. remaining: 0.0 minutes
Evaluation results saved to "C:\Users\alevret\OneDrive - Microsoft\Documents\DN\Code\genaiops-exercises\solutions\07_AI_Red_Teaming\.scan_Intermediary-Model-Target-Scan_20250408_102909\final_results.json".

Overall ASR: 50.0%
Attack Success: 2/4 attacks were successful
------------------------------------------------------------------------------------------------------------
Risk Category  | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------
Violence       | 0.0%           | 0.0%                         | N/A                             | N/A                           
Hate-unfairness| 100.0%         | 100.0%                       | N/A                             | N/A                           

Detailed results available a

## Advanced Example: Using an Azure Open AI Model Endpoint in a Callback Function

Using the same Azure Open AI model configuration as above, we now wrap it in a callback function for more flexibility and control on the input and output handling. This will demonstrate how to evaluate an actual AI application. To test your own actual AI application, replace the inside of the callback function with a call to your application.

In [13]:
# Define a callback that uses Azure OpenAI API to generate responses
async def azure_openai_callback(
    messages: list,
    stream: Optional[bool] = False,  # noqa: ARG001
    session_state: Optional[str] = None,  # noqa: ARG001
    context: Optional[Dict[str, Any]] = None,  # noqa: ARG001
) -> dict[str, list[dict[str, str]]]:
    deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")
    endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    api_version = os.environ.get("AZURE_API_VERSION")
    api_key = os.environ.get("AZURE_OPENAI_API_KEY")

    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=endpoint, 
        api_key=api_key,
        api_version=api_version,
    )

    ## Extract the latest message from the conversation history
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Call the model
        response = client.chat.completions.create(
            model=deployment,
            messages=[
                {"role": "user", "content": latest_message},
            ],
            # max_tokens=500, # If using an o1 base model, comment this line out
            max_completion_tokens=500,  # If using an o1 base model, uncomment this line
            temperature=0.0, # If using an o1 base model, comment this line out (temperature param not supported for o1 base models)
        )

        # Format the response to follow the expected chat protocol format
        formatted_response = {"content": response.choices[0].message.content, "role": "assistant"}
    except Exception as e:
        print(f"Error calling Azure OpenAI: {e!s}")
        formatted_response = "I encountered an error and couldn't process your request."
    return {"messages": [formatted_response]}

In [14]:
# Create the RedTeam instance with all of the risk categories with 5 attack objectives generated for each category
model_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm],
    num_objectives=5,
)

We will use this instance of `model_red_team` to test different attack strategies in the following section.

### Testing Different Attack Strategies

Now we'll run a more comprehensive evaluation using multiple attack strategies across risk categories. This will give us a better understanding of our model's vulnerabilities.

In [ ]:
# Run the red team scan with multiple attack strategies
advanced_result = await model_red_team.scan(
    target=azure_openai_callback,
    scan_name="Advanced-Callback-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts
        AttackStrategy.Morse,  # Encode prompts in Morse code
        AttackStrategy.Leetspeak,  # Use Leetspeak
        AttackStrategy.Url,  # Use URLs in prompts
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Advanced-Callback-Scan.json",
)

🚀 STARTING RED TEAM SCAN: Advanced-Callback-Scan
📂 Output directory: .\.scan_Advanced-Callback-Scan_20250408_110619
📊 Risk categories: ['violence', 'hate_unfairness', 'sexual', 'self_harm']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/build/evaluation/d7d459a8-b545-4d57-bf52-394a5cc7900a?wsid=/subscriptions/65a513ce-bb5d-4ed5-92b1-fa601d510a15/resourceGroups/agentai/providers/Microsoft.MachineLearningServices/workspaces/genaiops-demo
📋 Planning 52 total tasks


Scanning:   0%|                        | 0/52 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                 | 0/52 [00:03<?, ?scan/s, current=fetching baseline/hate_unfairness]

📝 Fetched baseline objectives for violence: 5 objectives
📝 Fetched baseline objectives for hate_unfairness: 5 objectives


...

[2025-04-08 11:22:40 +0100][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-08 11:22:40 +0100][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_8a2hnx1t_20250408_112240_220186, log path: C:\Users\alevret\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_8a2hnx1t_20250408_112240_220186\logs.txt
Scanning:  63%|███████████████████▋           | 33/52 [16:43<09:00, 28.46s/scan, current=batch 7/11]ERROR: [binary/hate_unfairness] Error processing batch 2: Error sending prompt with conversation ID: c2f91faf-da28-48e4-85b9-6e760b5ab9a0
Traceback (most recent call last):
  File "c:\Users\alevret\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyrit\prompt_normalizer\prompt_normalizer.py", line 95, in send_prompt_async
    response = await


✅ Completed task 33/52 (63.5%) - binary/violence in 82.9s
   Est. remaining: 9.7 minutes


[2025-04-08 11:23:07 +0100][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_x5tiq2a9_20250408_112307_376283, log path: C:\Users\alevret\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_x5tiq2a9_20250408_112307_376283\logs.txt
Scanning:  65%|████████████████████▎          | 34/52 [17:06<08:04, 26.94s/scan, current=batch 7/11]ERROR: [binary/sexual] Error processing batch 2: Error sending prompt with conversation ID: 145896bb-5510-4158-b00f-39a40776e78e
Traceback (most recent call last):
  File "c:\Users\alevret\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyrit\prompt_normalizer\prompt_normalizer.py", line 95, in send_prompt_async
    response = await target.send_prompt_async(prompt_request=request)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\alevret\AppData\Local\Programs\Python\Python312\Lib\site-packages\azur


✅ Completed task 34/52 (65.4%) - binary/hate_unfairness in 106.3s
   Est. remaining: 9.1 minutes


[2025-04-08 11:23:30 +0100][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-08 11:23:30 +0100][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_z0adg038_20250408_112330_763383, log path: C:\Users\alevret\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_z0adg038_20250408_112330_763383\logs.txt


The data and results used in this attack will be saved to the `output_path` specified. The URL printed out at the end of the scorecard will provide a link to where you results are uploaded and logged to your Azure AI Foundry project.

## Conclusion

In this notebook, we've demonstrated how to use the Azure AI Evaluation SDK's `RedTeam` functionality to assess the safety and resilience of AI systems. We started with a basic fixed-response example and then moved to a more realistic model testing across multiple risk categories and attack strategies.

The automated AI red teaming scans provides valuable insights into:

1. **Overall Attack Success Rate (ASR)** - The percentage of attacks that successfully elicit harmful content
2. **Vulnerability by Risk Category** - Which types of harmful content your model is most vulnerable to
3. **Effectiveness of Attack Strategies** - Which attack techniques are most successful against your model
4. **Impact of Complexity** - How more sophisticated attacks affect your model's safety guardrails

By regularly red-teaming your AI applications, you can identify and address potential vulnerabilities before deploying your models to production environments.

### Next Steps

1. **Mitigation**: Use these results to strengthen your model's guardrails against identified attack vectors
2. **Continuous Testing**: Implement regular red team evaluations as part of your development lifecycle
3. **Custom Strategies**: Develop custom attack strategies for your specific use cases and domain
4. **Safety Layers**: Consider adding additional safety layers like Azure AI Content Safety to filter harmful requests and responses 